In [3]:
import time
import neptune # type: ignore
import datetime
import torch
import torch.optim as optim
import torch.nn as nn

from torch.optim import SGD, Adam # type: ignore
from torch.optim.lr_scheduler import LambdaLR # type: ignore
from torch.utils.data import DataLoader
from torchvision import transforms
from torchvision.datasets import VOCSegmentation
from torchmetrics import Accuracy # type: ignore

from configs.utils import *
from datasets.utils import *
from datasets.utils import FlairDs
from easydict import EasyDict # type: ignore
from argparse import ArgumentParser, Namespace
from model.segmenter import Segmenter
from model.segmenter_adapt import SegmenterAdapt
from dl_toolbox.callbacks import EarlyStopping # type: ignore
from model.segmenter_adapt import *
from configs.utils import *
from datasets.utils import *
from easydict import EasyDict

/d/maboum/dil_seg/lib/python3.8/site-packages/albumentations/__init__.py:13: UserWarning: A new version of Albumentations is available: 1.4.15 (you have 1.4.14). Upgrade using: pip install -U albumentations. To disable automatic update checks, set the environment variable NO_ALBUMENTATIONS_UPDATE to 1.
  check_for_updates()


In [4]:
args = Namespace(
    initial_lr=0.01,
    final_lr=0.005,
    lr_milestones=(20, 80),
    epoch_len=100,
    sup_batch_size=4,
    crop_size=256,
    workers=6,
    img_aug='d4_rot90_rot270_rot180_d1flip',
    max_epochs=200,
    sequence_path="",
    train_split_coef=0.85,
    strategy='continual_{}',
    commit=None,
    train_type="adaptmlp",
    replay=False,  # Par défaut, `store_true` est `False`
    config_file= "/d/maboum/css-peft/configs/config.yml",
    ffn_adapt=True,  # Par défaut, `store_true` est `True`
    ffn_num=64,
    vpt=False,  # Par défaut, `store_true` est `False`
    vpt_num=1,
    fulltune=False  # Par défaut, `store_true` est `False`
)

tuning_config = EasyDict(
        # AdaptFormer
        ffn_adapt=args.ffn_adapt,
        ffn_option="parallel",
        ffn_adapter_layernorm_option="none",
        ffn_adapter_init_option="lora",
        ffn_adapter_scalar="0.1",
        ffn_num=args.ffn_num,
        d_model=768,
        # VPT related
        vpt_on=args.vpt,
        vpt_num=args.vpt_num,
        nb_task = 3, 
        decoder = "linear"
    )
config_file = "/d/maboum/css-peft/configs/config.yml"
config = load_config_yaml(file_path = config_file)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
dataset = config["dataset"]
data_config = dataset["flair1"]
seed = config["seed"]
directory_path = data_config["data_path"]
metadata = data_config["metadata"]
data_sequence = data_config["task_name"]
epochs = data_config['epochs']
eval_freq = data_config['eval_freq']
im_size = data_config["im_size"]
lr = data_config['learning_rate']
win_size = data_config["window_size"]
win_stride = data_config["window_stride"]
n_channels = data_config['n_channels']
n_class = 13
class_names = data_config["classnames_binary"]
eval_freq = data_config["eval_freq"]

selected_model = '_'.join([config["model_name"], "224"])
model = config["model"]
model_config = model[selected_model]
im_size = model_config["image_size"]
patch_size = model_config["patch_size"]
d_model = model_config["d_model"]
n_heads = model_config["n_heads"]
n_layers = model_config["n_layers"]
d_encoder = model_config["d_model"]

train_type = args.train_type
lora_params = config["lora_parameters"]
lora_rank = lora_params["rank"]
lora_alpha = lora_params["rank"]

binary = data_config["binary"]

In [5]:
seed = config["seed"]
torch.manual_seed(seed)
torch.cuda.manual_seed(seed)
torch.autograd.set_detect_anomaly(True) 
random.seed(seed)

In [6]:
run = neptune.init_run(
        project="continual-semantic-segmentation/peft-methods",
        api_token = "eyJhcGlfYWRkcmVzcyI6Imh0dHBzOi8vYXBwLm5lcHR1bmUuYWkiLCJhcGlfdXJsIjoiaHR0cHM6Ly9hcHAubmVwdHVuZS5haSIsImFwaV9rZXkiOiIwN2IzOGYxMC0xYTg5LTQxMGEtYjE3Yy1iNDVkZDM1MmEzYzIifQ==",
        name="AdaptFormerSeg",
        description="First run for Adapters project",
        tags=["adaptmlp", "test", "segmenter", "vit-large"],
        mode= "debug")

In [7]:
# Learning rate
def lambda_lr(epoch):
    m = epoch / args.max_epochs
    if m < args.lr_milestones[0]:
        return 1
    elif m < args.lr_milestones[1]:
        return 1 + ((m - args.lr_milestones[0]) / (
                    args.lr_milestones[1] - args.lr_milestones[0])) * (
                            args.final_lr / args.initial_lr - 1)
    else:
        return args.final_lr / args.initial_lr

In [8]:
# Callbacks
segmentation_model = Segmenter(im_size, n_layers, d_model, d_encoder, 4 * d_model, n_heads, n_class,
                                    patch_size, selected_model, model_name=config["model_name"], tuning_config=tuning_config).to(device)

segmentation_model.load_pretrained_weights()
segmentation_model_path = os.path.join("C:/Users/marie/css-peft/checkpoints",args.sequence_path.format(seed),
                                 '{}_{}'.format(args.strategy.format(train_type),seed))

optimizer = SGD(segmentation_model.parameters(),
                lr=args.initial_lr,
                momentum=0.9)
scheduler = LambdaLR(optimizer,lr_lambda= lambda_lr, verbose = True)
loss_fn = torch.nn.CrossEntropyLoss().cuda()
accuracy = Accuracy(task='multiclass',num_classes=n_class).cuda()

/d/maboum/dil_seg/lib/python3.8/site-packages/timm/models/_factory.py:114: UserWarning: Mapping deprecated model name vit_base_patch8_224_dino to current vit_base_patch8_224.dino.
  model = create_fn(


Pretrained model loaded successfully from timm!


/d/maboum/dil_seg/lib/python3.8/site-packages/torch/optim/lr_scheduler.py:28: UserWarning: The verbose parameter is deprecated. Please use get_last_lr() to access the learning rate.
  warnings.warn("The verbose parameter is deprecated. Please use get_last_lr() "


In [9]:
segmentation_model

Segmenter(
  (encoder): VisionTransformer(
    (patch_embed): PatchEmbedding(
      (proj): Conv2d(3, 768, kernel_size=(8, 8), stride=(8, 8))
    )
    (dropout): Dropout(p=0.1, inplace=False)
    (blocks): ModuleList(
      (0-11): 12 x Block(
        (norm1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (norm2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): Attention(
          (qkv): Linear(in_features=768, out_features=2304, bias=True)
          (attn_drop): Dropout(p=0.1, inplace=False)
          (proj): Linear(in_features=768, out_features=768, bias=True)
          (proj_drop): Dropout(p=0.1, inplace=False)
        )
        (mlp): FeedForward(
          (fc1): Linear(in_features=768, out_features=3072, bias=True)
          (act): GELU(approximate='none')
          (fc2): Linear(in_features=3072, out_features=768, bias=True)
          (drop): Dropout(p=0.1, inplace=False)
        )
        (drop_path): Identity()
      )
    )
    (norm): 

In [30]:
# Récupérer le dictionnaire des paramètres
params_dict = segmentation_model.encoder.blocks[0].adaptmlp_pool[0].state_dict()

params_dict

OrderedDict([('down_proj.weight',
              tensor([[ 0.0116, -0.0182,  0.0166,  ..., -0.0267,  0.0017, -0.0172],
                      [ 0.0001,  0.0232, -0.0199,  ...,  0.0158,  0.0006,  0.0153],
                      [-0.0394,  0.0052, -0.0093,  ..., -0.0301,  0.0264, -0.0039],
                      ...,
                      [ 0.0187,  0.0441,  0.0388,  ..., -0.0041, -0.0162, -0.0269],
                      [ 0.0391,  0.0049, -0.0219,  ..., -0.0109, -0.0306, -0.0196],
                      [-0.0197, -0.0351, -0.0327,  ..., -0.0196, -0.0114, -0.0259]],
                     device='cuda:0')),
             ('down_proj.bias',
              tensor([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
                      0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
                      0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
                     device='cuda:0')),

In [34]:
# Récupérer les poids et les biais
weights = params_dict['up_proj.weight']
biases = params_dict['up_proj.bias']

In [35]:
weights.shape, biases.shape

(torch.Size([768, 64]), torch.Size([768]))

In [ ]:
train_imgs, test_imgs = [],[]
for step,domain in enumerate(data_sequence[:2]):

    img = glob.glob(os.path.join(directory_path, '{}/Z*_*/img/IMG_*.tif'.format(domain)))
    random.shuffle(img)
    train_imgs += img[:int(len(img)*args.train_split_coef)]
    test_imgs += img[int(len(img)*args.train_split_coef):]
    random.shuffle(train_imgs)
    # Train&Validation Data
    domain_img_train = train_imgs[:int(len(train_imgs)*args.train_split_coef)]
    domain_img_val = train_imgs[int(len(train_imgs)*args.train_split_coef):]
    train_loader = create_train_dataloader(domain_img_train, args, data_config, binary= binary)
    val_loader = create_val_dataloader(domain_img_val, args, data_config, binary= binary)

    for param in segmentation_model.encoder.parameters():
        param.requires_grad = False
    # Unfreeze the adapt_mlp layers
    for name, param in segmentation_model.encoder.named_parameters():
        if 'adaptmlp' in name:
            param.requires_grad = True

    for epoch in range(1,args.max_epochs):
        time_ep = time.time()
        segmentation_model, train_acc, train_loss = train_function(segmentation_model,train_loader, 
                                            device,optimizer, loss_fn,
                                            accuracy, epoch, data_config, run)
        print(train_acc, train_loss)
        scheduler.step()
        segmentation_model, val_loss, val_acc = validation_function(segmentation_model,val_loader, 
                                                        device,loss_fn,
                                                        accuracy, epoch, data_config, run)
        print(val_acc, val_loss)
        early_stopping(val_loss,segmentation_model)
        if early_stopping.early_stop:
            break
        time_ep = time.time() - time_ep
    segmentation_model.increment()
